In [1]:
# Step 1: Load the Dataset
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('binary_class').getOrCreate()
df = spark.read.csv('classification_data.csv', inferSchema=True, header=True)

In [2]:
# Step 2: Explore the Dataframe
print((df.count(),len(df.columns)))

(46751, 12)


In [3]:
df.printSchema()

root
 |-- loan_id: string (nullable = true)
 |-- loan_purpose: string (nullable = true)
 |-- is_first_loan: integer (nullable = true)
 |-- total_credit_card_limit: integer (nullable = true)
 |-- avg_percentage_credit_card_limit_used_last_year: double (nullable = true)
 |-- saving_amount: integer (nullable = true)
 |-- checking_amount: integer (nullable = true)
 |-- is_employed: integer (nullable = true)
 |-- yearly_salary: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- dependent_number: integer (nullable = true)
 |-- label: integer (nullable = true)



In [4]:
df.show(5)

+-------+------------+-------------+-----------------------+-----------------------------------------------+-------------+---------------+-----------+-------------+---+----------------+-----+
|loan_id|loan_purpose|is_first_loan|total_credit_card_limit|avg_percentage_credit_card_limit_used_last_year|saving_amount|checking_amount|is_employed|yearly_salary|age|dependent_number|label|
+-------+------------+-------------+-----------------------+-----------------------------------------------+-------------+---------------+-----------+-------------+---+----------------+-----+
|    A_1|    personal|            1|                   7900|                                            0.8|         1103|           6393|          1|        16400| 42|               4|    0|
|    A_2|    personal|            0|                   3300|                                           0.29|         2588|            832|          1|        75500| 56|               1|    0|
|    A_3|    personal|            0|    

In [5]:
df.groupBy('label').count().show()

+-----+-----+
|label|count|
+-----+-----+
|    1|16201|
|    0|30550|
+-----+-----+



In [6]:
df.groupBy('loan_purpose').count().show()

+------------+-----+
|loan_purpose|count|
+------------+-----+
|      others| 6763|
|   emergency| 7562|
|    property|11388|
|  operations|10580|
|    personal|10458|
+------------+-----+



In [7]:
# Data Transformation
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
loan_purpose_indexer = StringIndexer(inputCol='loan_purpose',outputCol='loan_index_1').fit(df)
df = loan_purpose_indexer.transform(df)
loan_encoder = OneHotEncoder(inputCol='loan_index_1',outputCol='loan_purpose_vect').fit(df)
df = loan_encoder.transform(df)
df.select(['loan_purpose','loan_index_1','loan_purpose_vect']).show(3,False)

+------------+------------+-----------------+
|loan_purpose|loan_index_1|loan_purpose_vect|
+------------+------------+-----------------+
|personal    |2.0         |(4,[2],[1.0])    |
|personal    |2.0         |(4,[2],[1.0])    |
|personal    |2.0         |(4,[2],[1.0])    |
+------------+------------+-----------------+
only showing top 3 rows



In [8]:
from pyspark.ml.feature import VectorAssembler
df_assembler = VectorAssembler(inputCols= ['is_first_loan','total_credit_card_limit','avg_percentage_credit_card_limit_used_last_year',
                                          'saving_amount','checking_amount','is_employed','yearly_salary','age','dependent_number',
                                           'loan_purpose_vect'], outputCol='features')
df = df_assembler.transform(df)
df.select(['features','label']).show(10,False)

+--------------------------------------------------------------------+-----+
|features                                                            |label|
+--------------------------------------------------------------------+-----+
|[1.0,7900.0,0.8,1103.0,6393.0,1.0,16400.0,42.0,4.0,0.0,0.0,1.0,0.0] |0    |
|[0.0,3300.0,0.29,2588.0,832.0,1.0,75500.0,56.0,1.0,0.0,0.0,1.0,0.0] |0    |
|[0.0,7600.0,0.9,1651.0,8868.0,1.0,59000.0,46.0,1.0,0.0,0.0,1.0,0.0] |0    |
|[1.0,3400.0,0.38,1269.0,6863.0,1.0,26000.0,55.0,8.0,0.0,0.0,1.0,0.0]|0    |
|[0.0,2600.0,0.89,1310.0,3423.0,1.0,9700.0,41.0,4.0,0.0,0.0,0.0,1.0] |1    |
|[0.0,7600.0,0.51,1040.0,2406.0,1.0,22900.0,52.0,0.0,0.0,1.0,0.0,0.0]|0    |
|[1.0,6900.0,0.82,2408.0,5556.0,1.0,34800.0,48.0,4.0,0.0,1.0,0.0,0.0]|0    |
|[0.0,5700.0,0.56,1933.0,4139.0,1.0,32500.0,64.0,2.0,0.0,0.0,1.0,0.0]|0    |
|[1.0,3400.0,0.95,3866.0,4131.0,1.0,13300.0,23.0,3.0,0.0,0.0,1.0,0.0]|0    |
|[0.0,2900.0,0.91,88.0,2725.0,1.0,21100.0,52.0,1.0,0.0,0.0,1.0,0.0]  |1    |

In [9]:
model_df = df.select(['features','label'])

In [10]:
# Splittting into Train and Test Data
training_df,test_df=model_df.randomSplit([0.75,0.25])

In [11]:
# Model Training
from pyspark.ml.classification import LogisticRegression
log_reg = LogisticRegression().fit(training_df)
lr_summary = log_reg.summary
lr_summary.accuracy

0.8945759030635574

In [12]:
lr_summary.areaUnderROC

0.9592862324650032

In [13]:
print(lr_summary.precisionByLabel)

[0.9237842617152962, 0.8411736178467507]


In [14]:
predictions = log_reg.transform(test_df)
predictions.show(10)

+--------------------+-----+--------------------+--------------------+----------+
|            features|label|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|(13,[0,1,2,3,4,7]...|    1|[-6.9043316031559...|[0.00100242368176...|       1.0|
|(13,[0,1,2,3,4,7]...|    1|[-1.5292647391532...|[0.17810128869350...|       1.0|
|(13,[0,1,2,3,4,7,...|    1|[-6.0680728009845...|[0.00231028183444...|       1.0|
|(13,[0,1,2,3,4,7,...|    1|[-7.0359293413707...|[8.78927050751620...|       1.0|
|(13,[0,1,2,3,4,7,...|    1|[-6.6783219287199...|[0.00125630673464...|       1.0|
|(13,[0,1,2,3,4,7,...|    1|[-6.3616011657617...|[0.00172362391579...|       1.0|
|(13,[0,1,2,3,4,7,...|    1|[-5.0415327912804...|[0.00642232020890...|       1.0|
|(13,[0,1,2,3,4,7,...|    1|[-4.8881091666718...|[0.00747929626883...|       1.0|
|(13,[0,1,2,3,4,7,...|    1|[-6.0056400757136...|[0.00245875082130...|       1.0|
|(13,[0,1,2,3,4,

In [15]:
model_predictions = log_reg.transform(test_df)
model_predictions = log_reg.evaluate(test_df)
model_predictions.accuracy

0.891402330130113

In [16]:
model_predictions.areaUnderROC

0.9577655689243675

In [17]:
print(model_predictions.recallByLabel)

[0.9106515801794772, 0.855036855036855]


In [18]:
print(model_predictions.precisionByLabel)

[0.922286617492097, 0.8351331893448524]


In [ ]:
# Hyperparameter Tuning
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier()
rf_model = rf.fit(training_df)
model_predictions = rf_model.transform(test_df)

In [ ]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator()
rf = RandomForestClassifier()
paramGrid = (ParamGridBuilder().addGrid(rf.maxDepth,[5,10,20,25,30]).addGrid(rf.maxBins,[20,30,40]).addGrid(rf.numTrees,[5,20,50])
            .build())

In [ ]:
cv = CrossValidator(estimator=rf,estimatorParamMaps=paramGrid,evaluator=evaluator,numFolds=5)
cv_model = cv.fit(training_df)

In [ ]:
# Best Model
best_rf_model = cv_model.bestModel
model_predictions = best_rf_model.transform(test_df)
true_pos = model_predictions.filter(model_predictions['label'] == 1).filter(model_predictions['prediction'] == 1).count()
actual_pos = model_predictions.filter((model_predictions['label'] == 1)).count()
pred_pos = model_predictions.filter([model_predictions['prediction'] == 1]).count()
recall_rate = float(true_pos)/(actual_pos)
print(recall_rate)